# Project: Building a Proactive Tutoring Bot with a "Success Predictor" Model

**Goal:** Build a machine learning model that can predict a student's likelihood of success on their next problem. This "success probability" will serve as a real-time struggle metric to drive a proactive and intelligent tutoring system.

**Methodology:**
1.  **Data Foundation:** Load and clean data from ASSISTments, KDD Cup, and the large EdNet dataset by streaming directly from their respective data archives.
2.  **Canonical Skill Ontology:** Create a "master" list of skills using semantic embeddings to ensure that skills from different datasets are comparable.
3.  **Feature Engineering:** Create powerful, leakage-free historical features (`prior_is_correct`, `skill_correct_rate`, etc.).
4.  **Train & Validate:** Train a LightGBM "Success Predictor" model and validate its performance and generalization.
5.  **Inference at Scale:** Apply the trained model to the large EdNet dataset to demonstrate scalability.
6.  **Final Simulation:** Use the model to power a simulated tutor, showcasing its ability to provide multi-level, proactive support.

### Setup: Imports and Path Definitions
This cell imports all necessary libraries and defines the relative file paths for the portable project structure.

In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
import joblib
import io
import re
import torch
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer, util
import matplotlib.pyplot as plt
import seaborn as sns


# --- Define Relative Paths ---
# '..' goes UP one level from /notebooks/ to the main project root.
raw_data_dir = '../data/raw/'
output_dir = '../models/'
processed_data_dir = '../data/processed/'

# Paths to the two primary data archives
main_zip_path = os.path.join(raw_data_dir, 'Hint_Inference_Project_Data.zip')
ednet_zip_path = os.path.join(raw_data_dir, 'EdNet-KT1.zip')

# --- Create Output Directories ---
os.makedirs(output_dir, exist_ok=True)
os.makedirs(processed_data_dir, exist_ok=True)

print("Setup complete. Paths are defined.")

### Data Loading: Streaming from ZIP Archives
This section opens the data archives and streams the required CSV files directly into pandas DataFrames.

In [ ]:
print(f"Opening main dataset archive: {main_zip_path}")
try:
    main_zip_archive = zipfile.ZipFile(main_zip_path, 'r')
    
    print("Loading datasets directly from main ZIP file...")
    
    with main_zip_archive.open('assistments09.csv') as f:
        df_assistments = pd.read_csv(f, encoding='latin1', low_memory=False)

    with main_zip_archive.open('kdd_cup_2010_train.tsv') as f:
        df_kdd = pd.read_csv(f, sep='\\t')

    with main_zip_archive.open('ednet_questions.csv') as f:
        df_questions = pd.read_csv(f)

    print("ASSISTments, KDD, and EdNet Questions data loaded successfully.")

except FileNotFoundError:
    print(f"ERROR: Main ZIP file not found at '{main_zip_path}'! Please run the download script.")
    raise

### Data Cleaning and Artifact Creation
This section standardizes the schemas and creates two essential artifacts: the hint database and the canonical skill mapping.

In [ ]:
# =================================================================
# DATA CLEANING AND ARTIFACT CREATION
# =================================================================

# --- Schema Unification ---
def clean_response_time(series, upper_bound_sec=300, is_ms=False):
    series = pd.to_numeric(series, errors='coerce')
    if is_ms: series = series / 1000
    series = series.apply(lambda x: x if x > 0 else np.nan)
    series = series.apply(lambda x: min(x, upper_bound_sec) if pd.notna(x) else x)
    return series

df_assist_clean = df_assistments[['user_id', 'problem_id', 'skill_id', 'skill_name', 'correct', 'ms_first_response', 'hint_count']].copy()
df_assist_clean.rename(columns={'user_id': 'student_id', 'correct': 'is_correct', 'ms_first_response': 'response_time_sec'}, inplace=True)
df_assist_clean['response_time_sec'] = clean_response_time(df_assist_clean['response_time_sec'], is_ms=True)

df_kdd_clean = df_kdd[['Anon Student Id', 'Problem Name', 'KC(Default)', 'Correct First Attempt', 'Step Duration (sec)', 'Hints']].copy()
df_kdd_clean.rename(columns={'Anon Student Id': 'student_id', 'Problem Name': 'problem_id', 'KC(Default)': 'skill_id', 'Correct First Attempt': 'is_correct', 'Step Duration (sec)': 'response_time_sec', 'Hints': 'hint_count'}, inplace=True)
df_kdd_clean['response_time_sec'] = clean_response_time(df_kdd_clean['response_time_sec'], is_ms=False)
print("Schema unification complete.")


# --- Hint Database Creation (SOPHISTICATED VERSION) ---
print("Building structured hint database with varied, multi-level hints...")
simulated_hints_data = {
    'problem_id': [
        # Hints for a multi-step algebra problem
        'problem_algebra_1', 'problem_algebra_1', 'problem_algebra_1',
        # Hints for a geometry problem
        'problem_geometry_1', 'problem_geometry_1',
        # A simple hint for a basic problem
        'problem_arithmetic_1'
    ],
    'hint_level': [ 1, 2, 3, 1, 2, 1 ],
    'hint_text': [
        # Level 1 hint is conceptual
        "Hint 1/3: Remember to distribute the term outside the parentheses to both terms inside.",
        # Level 2 hint is procedural
        "Hint 2/3: After distributing, your equation should look like `2x + 6 = 12`. Now isolate 'x'.",
        # Level 3 hint is the direct answer
        "Hint 3/3 (Answer): The final answer is x=3.",

        # Level 1 hint provides the formula
        "Hint 1/2: The formula for the area of a triangle is (1/2) * base * height.",
        # Level 2 hint guides the next step
        "Hint 2/2: You have the base and area, so rearrange the formula to solve for the height.",

        # A simple check for a basic arithmetic problem
        "Hint 1/1: Don't forget to carry the one when you add the tens column."
    ]
}
df_hints = pd.DataFrame(simulated_hints_data)
hint_db_path = os.path.join(output_dir, 'structured_hints_database.csv')
df_hints.to_csv(hint_db_path, index=False)
print(f"Sophisticated hint database saved to {hint_db_path}")


# --- Canonical Skill Ontology Creation ---
print("\nBuilding canonical skill ontology...")
# ... (the rest of the code in this cell is correct and unchanged)
df_assist_skills = df_assistments[['skill_id', 'skill_name']].copy().dropna().drop_duplicates()
df_assist_skills = df_assist_skills[df_assist_skills['skill_name'] != '']
canonical_skills = df_assist_skills['skill_name'].unique()
kdd_raw_skills = df_kdd['KC(Default)'].dropna().unique()
kdd_skills = sorted(list(set([s.split('~~')[0].strip() for s in kdd_raw_skills])))
model = SentenceTransformer('all-MiniLM-L6-v2')
canonical_embeddings = model.encode(canonical_skills, convert_to_tensor=True)
kdd_embeddings = model.encode(kdd_skills, convert_to_tensor=True)
cos_scores = util.cos_sim(kdd_embeddings, canonical_embeddings)
kdd_to_assistments_map = {}
for i, kdd_skill_text in enumerate(kdd_skills):
    best_match_index = torch.argmax(cos_scores[i]).item()
    kdd_to_assistments_map[kdd_skill_text] = canonical_skills[best_match_index]
df_map1 = df_assist_skills.rename(columns={'skill_id': 'original_skill_id', 'skill_name': 'canonical_skill_name'})
df_map1['source'] = 'assistments'
df_map2 = pd.DataFrame(list(kdd_to_assistments_map.items()), columns=['original_skill_id', 'canonical_skill_name'])
df_map2['source'] = 'kdd'
df_skill_mapping = pd.concat([df_map1, df_map2], ignore_index=True)
mapping_path = os.path.join(output_dir, 'canonical_skill_mapping.csv')
df_skill_mapping.to_csv(mapping_path, index=False)
print(f"Canonical skill mapping saved.")

### Feature Engineering
This section creates the predictive features for our model using the canonical skill mapping.

In [ ]:
print("\nStarting Feature Engineering...")

df_assist_train = df_assist_clean.copy()
df_assist_train.rename(columns={'skill_id': 'original_skill_id'}, inplace=True)
df_assist_train['original_skill_id'] = df_assist_train['original_skill_id'].astype(str)
df_assist_final = pd.merge(df_assist_train, df_skill_mapping[df_skill_mapping['source'] == 'assistments'], on='original_skill_id', how='left')
df_kdd_train = df_kdd_clean.copy()
df_kdd_train.rename(columns={'skill_id': 'original_skill_id'}, inplace=True)
df_kdd_train['original_skill_id'] = df_kdd_train['original_skill_id'].astype(str).apply(lambda s: s.split('~~')[0].strip())
df_kdd_final = pd.merge(df_kdd_train, df_skill_mapping[df_skill_mapping['source'] == 'kdd'], on='original_skill_id', how='left')
df_train = pd.concat([df_assist_final, df_kdd_final], ignore_index=True)
df_train.dropna(subset=['canonical_skill_name'], inplace=True)
print(f"Created a unified training set with {len(df_train):,} mapped interactions.")

all_known_skills = list(df_train['canonical_skill_name'].unique())
all_known_skills.append('unknown_skill') 
canonical_skill_encoder = LabelEncoder().fit(all_known_skills)
df_train['skill_id_encoded'] = canonical_skill_encoder.transform(df_train['canonical_skill_name'])
encoder_path = os.path.join(output_dir, 'canonical_skill_encoder.joblib')
joblib.dump(canonical_skill_encoder, encoder_path)
print(f"Canonical skill encoder saved.")

print("Engineering historical features...")
df_train.sort_values(['student_id', 'response_time_sec'], inplace=True, kind='mergesort')
df_train['prior_is_correct'] = df_train.groupby('student_id')['is_correct'].shift(1)
df_train['prior_response_time'] = df_train.groupby('student_id')['response_time_sec'].shift(1)
df_train['skill_attempts'] = df_train.groupby(['student_id', 'skill_id_encoded']).cumcount()
skill_correct_sum = df_train.groupby(['student_id', 'skill_id_encoded'])['is_correct'].cumsum()
prior_correct_sum = skill_correct_sum - df_train['is_correct']
df_train['skill_correct_rate'] = np.where(df_train['skill_attempts'] > 0, prior_correct_sum / df_train['skill_attempts'], 0.5)
print("Feature engineering complete.")

### Model Training and Validation
We train the LightGBM model and validate its performance on a held-out portion of the training data.

In [ ]:
print("\nStarting Model Training and Validation...")

features = ['prior_response_time', 'prior_is_correct', 'skill_id_encoded', 'skill_attempts', 'skill_correct_rate']
target = 'is_correct'

df_modeling_data = df_train.dropna(subset=features + [target]).copy()
student_ids = df_modeling_data['student_id'].unique()
train_student_ids, val_student_ids = train_test_split(student_ids, test_size=0.2, random_state=42)
train_df = df_modeling_data[df_modeling_data['student_id'].isin(train_student_ids)].copy()
val_df = df_modeling_data[df_modeling_data['student_id'].isin(val_student_ids)].copy()

X_train = train_df[features]
y_train = train_df[target]
X_val = val_df[features]
y_val = val_df[target]
print(f"Training on {len(X_train):,} interactions, validating on {len(X_val):,}.")

lgbm_success_predictor = lgb.LGBMClassifier(objective='binary', metric='auc', random_state=42)
lgbm_success_predictor.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(10, verbose=False)])

y_pred_proba = lgbm_success_predictor.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred_proba)
print(f"Model AUC on validation set: {auc:.4f}")

val_df['prob_of_success'] = y_pred_proba
val_df['actual_hint_count'] = val_df['hint_count'].clip(upper=4)
avg_prob_by_hint_count = val_df.groupby('actual_hint_count')['prob_of_success'].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(data=avg_prob_by_hint_count, x='actual_hint_count', y='prob_of_success', hue='actual_hint_count', palette='coolwarm_r', legend=False)
plt.title('Model Correctly Predicts Lower Success for Higher Hint Needs', fontsize=16)
plt.ylabel("Model's Average Predicted Probability of Success")
plt.xlabel('Actual Hint Count (Ground Truth for Struggle)')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

model_path = os.path.join(output_dir, 'lgbm_success_predictor.joblib')
joblib.dump(lgbm_success_predictor, model_path)
print(f"Trained Success Predictor model saved.")

### External Validation on a Hold-Out Dataset
We test the model on the entirely new `bridge_to_algebra` dataset.

In [ ]:
print("\nStarting External Validation...")

bridge_data_filename_in_zip = 'bridge_to_algebra_2006_train.txt'

try:
    print(f"Loading hold-out data '{bridge_data_filename_in_zip}' from archive...")
    with main_zip_archive.open(bridge_data_filename_in_zip) as f:
        df_bridge = pd.read_csv(f, sep='\\t', engine='python')
    print(f"Loaded {len(df_bridge):,} interactions from the hold-out set.")

    df_bridge_clean = df_bridge[['Anon Student Id', 'Problem Name', 'KC(SubSkills)', 'Correct First Attempt', 'Step Duration (sec)', 'Hints']].copy()
    df_bridge_clean.rename(columns={'Anon Student Id': 'student_id', 'Problem Name': 'problem_id', 'KC(SubSkills)': 'original_skill_id', 'Correct First Attempt': 'is_correct', 'Step Duration (sec)': 'response_time_sec', 'Hints': 'hint_count'}, inplace=True)
    df_bridge_clean.dropna(subset=['original_skill_id'], inplace=True)
    df_bridge_clean['original_skill_id'] = df_bridge_clean['original_skill_id'].astype(str).apply(lambda s: s.split('~~')[0].strip())
    df_bridge_mapped = pd.merge(df_bridge_clean, df_skill_mapping[df_skill_mapping['source'] == 'kdd'], on='original_skill_id', how='left').dropna(subset=['canonical_skill_name'])
    print(f"Successfully mapped {len(df_bridge_mapped):,} hold-out interactions.")

    df_bridge_mapped['skill_id_encoded'] = canonical_skill_encoder.transform(df_bridge_mapped['canonical_skill_name'])
    df_bridge_mapped.sort_values(['student_id', 'response_time_sec'], inplace=True, kind='mergesort')
    df_bridge_mapped['prior_is_correct'] = df_bridge_mapped.groupby('student_id')['is_correct'].shift(1)
    df_bridge_mapped['prior_response_time'] = df_bridge_mapped.groupby('student_id')['response_time_sec'].shift(1)
    df_bridge_mapped['skill_attempts'] = df_bridge_mapped.groupby(['student_id', 'skill_id_encoded']).cumcount()
    skill_correct_sum = df_bridge_mapped.groupby(['student_id', 'skill_id_encoded'])['is_correct'].cumsum()
    prior_correct_sum = skill_correct_sum - df_bridge_mapped['is_correct']
    df_bridge_mapped['skill_correct_rate'] = np.where(df_bridge_mapped['skill_attempts'] > 0, prior_correct_sum / df_bridge_mapped['skill_attempts'], 0.5)
    df_bridge_validation = df_bridge_mapped.dropna(subset=features).copy()
    
    if not df_bridge_validation.empty:
        print(f"Making predictions on {len(df_bridge_validation):,} hold-out interactions...")
        X_bridge_val = df_bridge_validation[features]
        df_bridge_validation['prob_of_success'] = lgbm_success_predictor.predict_proba(X_bridge_val)[:, 1]
        df_bridge_validation['actual_hint_count'] = df_bridge_validation['hint_count'].clip(upper=4)
        bridge_val_results = df_bridge_validation.groupby('actual_hint_count')['prob_of_success'].mean().reset_index()
        plt.figure(figsize=(10, 6))
        sns.barplot(data=bridge_val_results, x='actual_hint_count', y='prob_of_success', hue='actual_hint_count', palette='coolwarm_r', legend=False)
        plt.title('Validation on Hold-Out Data: Model Generalizes Well', fontsize=14)
        plt.ylabel("Model's Predicted Probability of Success")
        plt.xlabel('Actual Hint Count (Ground Truth Complexity)')
        plt.ylim(0, 1)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()
    else:
        print("No interactions remained in the hold-out set after processing.")

except FileNotFoundError:
    print(f"ERROR: Hold-out data '{bridge_data_filename_in_zip}' not found inside '{main_zip_path}'. Skipping.")

### Inference at Scale with EdNet
This step applies the model to the large `EdNet-KT1.zip` dataset. It will only run if you have manually placed this file in the `data/raw/` directory.

In [ ]:
print("\n--- Starting EdNet Inference (Optional) ---")

if not os.path.exists(ednet_zip_path):
    print(f"INFO: Large dataset '{ednet_zip_path}' not found. Skipping EdNet processing.")
    df_ednet_processed = pd.DataFrame() 
else:
    print("Large EdNet dataset found. Starting processing...")
    
    unknown_skill_id = canonical_skill_encoder.transform(['unknown_skill'])[0]
    MAX_STUDENTS_TO_PROCESS = 20000 
    MIN_INTERACTIONS_PER_STUDENT = 20
    all_processed_students = []
    students_processed_count = 0
    
    with zipfile.ZipFile(ednet_zip_path, 'r') as ednet_archive:
        student_files = [f for f in ednet_archive.namelist() if f.startswith('KT1/') and f.endswith('.csv')]
        print(f"Processing up to {MAX_STUDENTS_TO_PROCESS} students from EdNet-KT1...")
        for filename in student_files:
            if students_processed_count >= MAX_STUDENTS_TO_PROCESS: break
            
            user_id = os.path.splitext(os.path.basename(filename))[0]
            with ednet_archive.open(filename) as f:
                student_df = pd.read_csv(io.TextIOWrapper(f, 'utf-8'))
            
            if len(student_df) < MIN_INTERACTIONS_PER_STUDENT: continue
            students_processed_count += 1
            if students_processed_count > 0 and students_processed_count % 4000 == 0: print(f"  ...processed {students_processed_count} students...")

            processed_df = pd.merge(student_df, df_questions, on='question_id', how='left')
            processed_df['is_correct'] = (processed_df['user_answer'] == processed_df['correct_answer']).astype(int)
            processed_df.rename(columns={'elapsed_time': 'response_time_sec'}, inplace=True)
            processed_df['response_time_sec'] = clean_response_time(processed_df['response_time_sec'], is_ms=True)
            processed_df['user_id'] = user_id
            processed_df.sort_values('timestamp', inplace=True)

            processed_df['prior_is_correct'] = processed_df.groupby('user_id')['is_correct'].shift(1)
            processed_df['prior_response_time'] = processed_df.groupby('user_id')['response_time_sec'].shift(1)
            processed_df['skill_id_encoded'] = unknown_skill_id
            processed_df['skill_attempts'] = 0
            processed_df['skill_correct_rate'] = 0.5
            
            processed_df.dropna(subset=features, inplace=True)
            if processed_df.empty: continue

            processed_df['prob_of_success'] = lgbm_success_predictor.predict_proba(processed_df[features])[:, 1]
            final_student_df = processed_df[['user_id', 'question_id', 'is_correct', 'response_time_sec', 'prob_of_success']]
            all_processed_students.append(final_student_df)

    if all_processed_students:
        df_ednet_processed = pd.concat(all_processed_students, ignore_index=True)
        ednet_output_path = os.path.join(processed_data_dir, 'ednet_with_success_prob.csv')
        df_ednet_processed.to_csv(ednet_output_path, index=False, float_format='%.4f')
        print(f"EdNet processing complete. Saved {len(df_ednet_processed):,} interactions to {ednet_output_path}")
    else:
        print("No EdNet students met the processing criteria.")
        df_ednet_processed = pd.DataFrame()

### Final Dataset Unification
This step combines the original training data with the newly-processed EdNet data to create a single master dataset.

In [ ]:
print("\n--- Starting Final Dataset Unification ---")

# Use the data that went into the model
df_train_engineered = df_modeling_data.copy()
df_train_engineered['prob_of_success'] = lgbm_success_predictor.predict_proba(df_train_engineered[features])[:, 1]

# Standardize and Combine
df1 = df_train_engineered.copy()
df1.rename(columns={'student_id': 'user_id'}, inplace=True)
df1['source_dataset'] = 'assistments_kdd'
df1['actual_hint_count'] = df1['hint_count'].clip(upper=4)

if not df_ednet_processed.empty:
    df2 = df_ednet_processed.copy()
    df2.rename(columns={'question_id': 'problem_id'}, inplace=True)
    df2['source_dataset'] = 'ednet'
    df2['actual_hint_count'] = pd.NA
else:
    df2 = pd.DataFrame() 

final_columns = ['user_id', 'problem_id', 'is_correct', 'response_time_sec', 'prob_of_success', 'source_dataset', 'actual_hint_count']

if not df2.empty:
    df_master = pd.concat([df1[final_columns], df2[final_columns]], ignore_index=True)
else:
    df_master = df1[final_columns]
    
master_output_path = os.path.join(processed_data_dir, 'final_master_dataset.csv')
df_master.to_csv(master_output_path, index=False, float_format='%.4f')
print(f"SUCCESS! Final Master dataset with {len(df_master):,} interactions saved to: {master_output_path}")

### Final "Showcase" Simulation
This section demonstrates the complete system using real data points to create a compelling, multi-level interaction story.

In [ ]:
# =================================================================
# FINAL SHOWCASE SIMULATION
# =================================================================
print("\n--- Starting Showcase Simulation ---")

# Step 1: Define Tutor Logic
def get_tutor_action(student_features_df, problem_id, attempt_number):
    prob_success = lgbm_success_predictor.predict_proba(student_features_df[features])[:, 1][0]
    tutor_response = f"  (Model predicts a {prob_success:.1%} chance of success... "
    action_text = ""
    
    if prob_success < 0.45:
        tutor_response += "Intervention: HIGH STRUGGLE)"
        hint_row = df_hints[(df_hints['problem_id'] == problem_id) & (df_hints['hint_level'] == attempt_number)]
        
        # THIS IS THE FIX: The hint text is now correctly retrieved and assigned
        if not hint_row.empty:
            action_text = hint_row['hint_text'].iloc[0]
        else:
            action_text = f"Fallback Hint (Level {attempt_number})"
            
    elif prob_success < 0.65:
        tutor_response += "Intervention: MILD STRUGGLE)"
        action_text = "Passive Offer: Looks like this might be tricky. A hint is available if you need it."
        
    else:
        tutor_response += "Intervention: NONE)"
        action_text = "No action taken. Keep up the great work!"
        
    print(tutor_response)
    # This print statement now correctly displays the action/hint text
    print(f"Tutor Bot Says: {action_text}\n")
    

# Step 2: The "Showcase" Simulation using Real Data
print("\n--- Showcase Simulation: A Student's Learning Journey ---")

def find_real_student_state(target_prob_min, target_prob_max):
    if 'prob_of_success' not in val_df.columns:
         val_df['prob_of_success'] = lgbm_success_predictor.predict_proba(val_df[features])[:, 1]
    candidates = val_df[(val_df['prob_of_success'] >= target_prob_min) & (val_df['prob_of_success'] < target_prob_max)]
    return candidates.iloc[[0]][features] if not candidates.empty else None

# "Casting Call": Find actors for the play
state_confident = find_real_student_state(0.70, 1.0)
state_mild_struggle = find_real_student_state(0.50, 0.65)
state_high_struggle = find_real_student_state(0.0, 0.45)

if state_confident is None or state_mild_struggle is None or state_high_struggle is None:
    print("Could not find real student examples for all required states. Simulation cannot proceed.")
else:
    print("Found real student data to represent each state of the journey.\n")
    
    print("--- Act I: A Confident Start ---")
    get_tutor_action(state_confident, "problem_arithmetic_1", 1)
    
    print("--- Act II: The First Stumble ---")
    get_tutor_action(state_mild_struggle, "problem_geometry_1", 1)
    
    print("--- Act III: Deep Struggle & Proactive Hints ---")
    get_tutor_action(state_high_struggle, "problem_algebra_1", 1)
    
    print("--- Act IV: Re-attempting While Still Struggling ---")
    get_tutor_action(state_high_struggle, "problem_algebra_1", 2)
    
    print("--- Act V: The Recovery ---")
    get_tutor_action(state_confident, "problem_arithmetic_1", 1)

# --- Close the main zip file at the very end of the notebook ---
main_zip_archive.close()
print("\nNotebook execution complete. Main ZIP archive closed.")